**Clone And Install Dependencies**

In [10]:
# Clone the repository (if you haven't already in a fresh session)
!git clone https://github.com/Yui007/weebcentral_downloader

# Install all standard dependencies
!pip install requests beautifulsoup4 selenium tqdm IPython ipywidgets

# --- CRITICAL FIXES FOR SELENIUM ---
# 1. Install the auto-driver manager package
!pip install webdriver-manager

# 2. Install the necessary Chrome browser binary for Linux/Colab
!apt update && apt install -y chromium-browser

fatal: destination path 'weebcentral_downloader' already exists and is not an empty directory.
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,151 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,476 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-d

***Run The Scraper***

In [11]:
%cd /content/weebcentral_downloader
!python weebcentral_scraper_colab.py

/content/weebcentral_downloader
<IPython.core.display.HTML object>
Enter manga URL: https://weebcentral.com/series/01J76XYE63398XQW85ES97162W/Remina

Chapter Selection:
1. All chapters
2. Single chapter
3. Chapter range
Enter your choice (1-3): 2
Enter chapter number: 1

Enter output directory (default: manga_downloads): 
Enter delay between chapters in seconds (default: 1.0): 
Enter maximum number of download threads (default: 4): 
Convert chapters to CBZ? (y/n, default: n): y
Delete images after conversion? (y/n, default: n): 
2025-11-24 07:41:50 - INFO - Starting to scrape manga from: https://weebcentral.com/series/01J76XYE63398XQW85ES97162W/Remina
2025-11-24 07:41:51 - INFO - Manga title: Remina
2025-11-24 07:41:51 - INFO - Fetching chapter list from: https://weebcentral.com/series/01J76XYE63398XQW85ES97162W/full-chapter-list
2025-11-24 07:41:51 - INFO - Will download 1 chapters
2025-11-24 07:41:51 - INFO - Downloading chapter: Chapter 1
2025-11-24 07:41:51 - INFO - Loading page wi

**Zip The Manga Folder for Easy Download**

In [ ]:
# Add this as a new cell in your notebook
import os
import shutil

# Replace 'manga_downloads' with your output directory name if different
output_dir = '/content/weebcentral_downloader/manga_downloads'

if os.path.exists(output_dir):
    # Get all manga folders
    manga_folders = [d for d in os.listdir(output_dir)
                    if os.path.isdir(os.path.join(output_dir, d))]

    if manga_folders:
        print("Available manga folders:")
        for i, folder in enumerate(manga_folders, 1):
            print(f"{i}. {folder}")

        # Let user choose which folder to zip
        choice = int(input("\nEnter the number of the folder to zip (1, 2, etc.): ")) - 1
        if 0 <= choice < len(manga_folders):
            folder_name = manga_folders[choice]
            folder_path = os.path.join(output_dir, folder_name)
            zip_path = f"/content/{folder_name}.zip"

            print(f"\nCreating zip file for {folder_name}...")
            shutil.make_archive(f"/content/{folder_name}", 'zip', folder_path)

            print(f"\nZip file created at: {zip_path}")
            print("You can now download it manually from the Colab file browser.")
        else:
            print("Invalid choice!")
    else:
        print("No manga folders found!")
else:
    print(f"Directory {output_dir} not found!")

**If File Size Is Bigger Than 200MB and If You Want To Download In Chunkz**

In [ ]:
# Fast downloading using splits
import os
import math

# Replace 'manga_downloads' with your output directory name if different
output_dir = '/content/weebcentral_downloader/manga_downloads'

# Size of each split in MB
SPLIT_SIZE_MB = 200  # Adjust this value as needed

if os.path.exists(output_dir):
    # Get all manga folders
    manga_folders = [d for d in os.listdir(output_dir)
                    if os.path.isdir(os.path.join(output_dir, d))]

    if manga_folders:
        print("Available manga folders:")
        for i, folder in enumerate(manga_folders, 1):
            print(f"{i}. {folder}")

        # Let user choose which folder to split
        choice = int(input("\nEnter the number of the folder to split (1, 2, etc.): ")) - 1
        if 0 <= choice < len(manga_folders):
            folder_name = manga_folders[choice]
            folder_path = os.path.join(output_dir, folder_name)

            print(f"\nCreating split archives for {folder_name}...")

            # Create splits using 7z
            # -v200m : split into 200MB chunks (adjust size as needed)
            # -mx=1 : fastest compression
            # -mmt=on : multi-threading
            !7z a -v{SPLIT_SIZE_MB}m -mx=1 -mmt=on "/content/{folder_name}.7z" "{folder_path}"/*

            # Count the number of splits
            splits = [f for f in os.listdir('/content') if f.startswith(f"{folder_name}.7z")]

            print(f"\nCreated {len(splits)} split files at /content/:")
            for split in splits:
                size_mb = os.path.getsize(f"/content/{split}") / (1024 * 1024)
                print(f"- {split} ({size_mb:.1f} MB)")

            print("\nYou can now download these splits from the Colab file browser.")
            print("After downloading, combine them using 7-Zip on your PC.")
        else:
            print("Invalid choice!")
    else:
        print("No manga folders found!")
else:
    print(f"Directory {output_dir} not found!")